In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import pandas as pd
import numpy as np
import cv2 as cv
import PIL
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

DATA_LOC = r'C:\Temp\Data\state-farm-distracted-driver-detection'
CSV_LOC = os.path.join(DATA_LOC, 'driver_imgs_list.csv')
IMGS_LOC = os.path.join(DATA_LOC, 'imgs')
TRAIN_LOC = os.path.join(IMGS_LOC, 'train')
VAL_LOC = os.path.join(IMGS_LOC, 'val')
TEST_LOC = os.path.join(IMGS_LOC, 'test')
MODEL_LOC = lambda model_: r'C:\temp\Data\models\{}.model'.format(model_name)
OUT_LOC = os.path.join(DATA_LOC, 'submission.csv')

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
if True:
    # split data into train/test
    data = pd.read_csv(CSV_LOC)
    from sklearn.model_selection import train_test_split
    train, val = train_test_split(data, test_size=0.2)
if False: #if they don't exist
    # create val dir
    os.mkdir(os.path.join(IMGS_LOC, 'val'))
    for n in range(10):
        os.mkdir(os.path.join(IMGS_LOC, 'val', 'c'+str(n)))
if False:
    import shutil
    #move files from trian dir to val dir
    for src, dst in [(os.path.join(TRAIN_LOC, cn, img), os.path.join(VAL_LOC, cn, img))
                     for cn, img in val.loc[:,['classname','img']].values.tolist()]:
        if os.path.exists(src):
            shutil.move(src,dst)

In [4]:
total_train = train.shape[0]
total_val = val.shape[0]

In [5]:
batch_size = 40
EPOCHS = 50
IMG_HEIGHT = 480
IMG_WIDTH = 640
MODEL_NAME = 'sgd100nologits'

In [6]:
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=.15,  
    height_shift_range=.15,
    horizontal_flip=True,
    zoom_range=0.5
)

validation_image_generator = ImageDataGenerator(rescale=1./255)

In [7]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=TRAIN_LOC,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

Found 17939 images belonging to 10 classes.


In [8]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=VAL_LOC,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

Found 4485 images belonging to 10 classes.


In [9]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [10]:
if False:
    sample_training_images, _ = next(train_data_gen)
    plotImages(sample_training_images[:5])

In [11]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

In [12]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [13]:
# model = tf.keras.models.load_model(MODEL_LOC(MODEL_NAME))
# epochs = 21

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 480, 640, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 240, 320, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 240, 320, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 240, 320, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 120, 160, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 120, 160, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 160, 64)      1

In [ ]:
history = model.fit(
    train_data_gen,
    epochs=EPOCHS,
    validation_data=val_data_gen
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 449 steps, validate for 113 steps
Epoch 1/100
449/449 [==============================] - 950s 2s/step - loss: 2.3015 - accuracy: 0.1089 - val_loss: 2.3008 - val_accuracy: 0.1077
Epoch 2/100
449/449 [==============================] - 949s 2s/step - loss: 2.2962 - accuracy: 0.1136 - val_loss: 2.2770 - val_accuracy: 0.1530
Epoch 3/100
449/449 [==============================] - 948s 2s/step - loss: 2.2834 - accuracy: 0.1302 - val_loss: 2.2661 - val_accuracy: 0.1766
Epoch 4/100
389/449 [========================>.....] - ETA: 2:04 - loss: 2.2678 - accuracy: 0.1449

In [ ]:
list(history.history.keys())

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save(MODEL_LOC(MODEL_NAME))

In [ ]:
test_image_generator = ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                         directory=TEST_LOC,
                                                         target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                         class_mode='categorical')

In [ ]:
predict = model.predict(test_data_gen)

In [ ]:
len(predict)

In [ ]:
submission = pd.DataFrame([[fname]+ pred for fname, pred in (zip(os.listdir(os.path.join(TEST_LOC,'unknown')), predict.tolist()))], columns=['img'] + [f'c{n}' for n in range(10)])
submission

In [ ]:
submission.to_csv(OUT_LOC, index=False)